In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import animation
import time
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

2023-07-02 18:11:26.146048: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 18:11:26.267689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 18:11:26.268782: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 18:11:27.109255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
PATH = "~/Documents/data/kaggle-google-sign-language/"

In [3]:
df_train = pd.read_csv(f'{PATH}train.csv')

In [4]:
df_train.head()

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road


In [5]:
print("The longest phrase", df_train.phrase.str.len().max())
df_train[df_train.phrase.str.len() == df_train.phrase.str.len().max()]

The longest phrase 31


,path,file_id,sequence_id,participant_id,phrase
588,train_landmarks/5414471.parquet,5414471,1834593138,24,north charleston south carolina
2544,train_landmarks/128822441.parquet,128822441,214470015,56,north charleston south carolina
8136,train_landmarks/296317215.parquet,296317215,1499002901,147,north charleston south carolina
19204,train_landmarks/546816846.parquet,546816846,1461804071,109,north charleston south carolina
23293,train_landmarks/649779897.parquet,649779897,1272752879,102,north charleston south carolina
24146,train_landmarks/649779897.parquet,649779897,1300641797,13,north charleston south carolina
24880,train_landmarks/654436541.parquet,654436541,1674627414,81,north charleston south carolina
35436,train_landmarks/1134756332.parquet,1134756332,138576800,181,north charleston south carolina
35765,train_landmarks/1134756332.parquet,1134756332,149769917,223,north charleston south carolina
42189,train_landmarks/1365772051.parquet,1365772051,1942906736,176,north charleston south carolina


In [6]:
df = pd.read_parquet(PATH + df_train.path[0], engine='auto')
print(df.shape)
df.head()

(162699, 1630)


,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,x_face_10,x_face_11,x_face_12,x_face_13,x_face_14,x_face_15,x_face_16,x_face_17,x_face_18,x_face_19,x_face_20,x_face_21,x_face_22,x_face_23,x_face_24,x_face_25,x_face_26,x_face_27,x_face_28,x_face_29,x_face_30,x_face_31,x_face_32,x_face_33,x_face_34,x_face_35,x_face_36,x_face_37,x_face_38,x_face_39,x_face_40,x_face_41,x_face_42,x_face_43,x_face_44,x_face_45,x_face_46,x_face_47,x_face_48,x_face_49,x_face_50,x_face_51,x_face_52,x_face_53,x_face_54,x_face_55,x_face_56,x_face_57,x_face_58,x_face_59,x_face_60,x_face_61,x_face_62,x_face_63,x_face_64,x_face_65,x_face_66,x_face_67,x_face_68,x_face_69,x_face_70,x_face_71,x_face_72,x_face_73,x_face_74,x_face_75,x_face_76,x_face_77,x_face_78,x_face_79,x_face_80,x_face_81,x_face_82,x_face_83,x_face_84,x_face_85,x_face_86,x_face_87,x_face_88,x_face_89,x_face_90,x_face_91,x_face_92,x_face_93,x_face_94,x_face_95,x_face_96,x_face_97,x_face_98,x_face_99,x_face_100,x_face_101,x_face_102,x_face_103,x_face_104,x_face_105,x_face_106,x_face_107,x_face_108,x_face_109,x_face_110,x_face_111,x_face_112,x_face_113,x_face_114,x_face_115,x_face_116,x_face_117,x_face_118,x_face_119,x_face_120,x_face_121,x_face_122,x_face_123,x_face_124,x_face_125,x_face_126,x_face_127,x_face_128,x_face_129,x_face_130,x_face_131,x_face_132,x_face_133,x_face_134,x_face_135,x_face_136,x_face_137,x_face_138,x_face_139,x_face_140,x_face_141,x_face_142,x_face_143,x_face_144,x_face_145,x_face_146,x_face_147,x_face_148,x_face_149,x_face_150,x_face_151,x_face_152,x_face_153,x_face_154,x_face_155,x_face_156,x_face_157,x_face_158,x_face_159,x_face_160,x_face_161,x_face_162,x_face_163,x_face_164,x_face_165,x_face_166,x_face_167,x_face_168,x_face_169,x_face_170,x_face_171,x_face_172,x_face_173,x_face_174,x_face_175,x_face_176,x_face_177,x_face_178,x_face_179,x_face_180,x_face_181,x_face_182,x_face_183,x_face_184,x_face_185,x_face_186,x_face_187,x_face_188,x_face_189,x_face_190,x_face_191,x_face_192,x_face_193,x_face_194,x_face_195,x_face_196,x_face_197,x_face_198,x_face_199,x_face_200,x_face_201,x_face_202,x_face_203,x_face_204,x_face_205,x_face_206,x_face_207,x_face_208,x_face_209,x_face_210,x_face_211,x_face_212,x_face_213,x_face_214,x_face_215,x_face_216,x_face_217,x_face_218,x_face_219,x_face_220,x_face_221,x_face_222,x_face_223,x_face_224,x_face_225,x_face_226,x_face_227,x_face_228,x_face_229,x_face_230,x_face_231,x_face_232,x_face_233,x_face_234,x_face_235,x_face_236,x_face_237,x_face_238,x_face_239,x_face_240,x_face_241,x_face_242,x_face_243,x_face_244,x_face_245,x_face_246,x_face_247,x_face_248,...,z_face_293,z_face_294,z_face_295,z_face_296,z_face_297,z_face_298,z_face_299,z_face_300,z_face_301,z_face_302,z_face_303,z_face_304,z_face_305,z_face_306,z_face_307,z_face_308,z_face_309,z_face_310,z_face_311,z_face_312,z_face_313,z_face_314,z_face_315,z_face_316,z_face_317,z_face_318,z_face_319,z_face_320,z_face_321,z_face_322,z_face_323,z_face_324,z_face_325,z_face_326,z_face_327,z_face_328,z_face_329,z_face_330,z_face_331,z_face_332,z_face_333,z_face_334,z_face_335,z_face_336,z_face_337,z_face_338,z_face_339,z_face_340,z_face_341,z_face_342,z_face_343,z_face_344,z_face_345,z_face_346,z_face_347,z_face_348,z_face_349,z_face_350,z_face_351,z_face_352,z_face_353,z_face_354,z_face_355,z_face_356,z_face_357,z_face_358,z_face_359,z_face_360,z_face_361,z_face_362,z_face_363,z_face_364,z_face_365,z_face_366,z_face_367,z_face_368,z_face_369,z_face_370,z_face_371,z_face_372,z_face_373,z_face_374,z_face_375,z_face_376,z_face_377,z_face_378,z_face_379,z_face_380,z_face_381,z_face_382,z_face_383,z_face_384,z_face_385,z_face_386,z_face_387,z_face_388,z_face_389,z_face_390,z_face_391,z_face_392,z_face_393,z_face_394,z_face_395,z_face_396,z_face_397,z_face_398,z_face_399,z_face_400,z_face_401,z_face_402,z_face_403,z_face_404,z_face_405,z_face_406,z_face_407,z_face_408,z_face_409,z_face_410,z_face_411,z_face_412,z_face_413,z_face_414,z_face_415,z_face_

In [7]:
longest_sequence = 0
for path in tqdm(df_train.path.unique()):
    df = pd.read_parquet(PATH + df_train.path[0], engine="auto")
    sequence_frames = df.groupby("sequence_id").frame.count().max()
    if sequence_frames > longest_sequence:
        longest_sequence = sequence_frames
print(longest_sequence)        

100%|██████████████████████████████████████████████████████████████████████| 68/68 [01:09<00:00,  1.02s/it]

672


In [8]:
spots = ['face', 'pose', 'left_hand', 'right_hand']

In [9]:
df = pd.read_parquet(PATH + "train_landmarks/5414471.parquet", engine='auto')
df.head()

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,x_face_10,x_face_11,x_face_12,x_face_13,x_face_14,x_face_15,x_face_16,x_face_17,x_face_18,x_face_19,x_face_20,x_face_21,x_face_22,x_face_23,x_face_24,x_face_25,x_face_26,x_face_27,x_face_28,x_face_29,x_face_30,x_face_31,x_face_32,x_face_33,x_face_34,x_face_35,x_face_36,x_face_37,x_face_38,x_face_39,x_face_40,x_face_41,x_face_42,x_face_43,x_face_44,x_face_45,x_face_46,x_face_47,x_face_48,x_face_49,x_face_50,x_face_51,x_face_52,x_face_53,x_face_54,x_face_55,x_face_56,x_face_57,x_face_58,x_face_59,x_face_60,x_face_61,x_face_62,x_face_63,x_face_64,x_face_65,x_face_66,x_face_67,x_face_68,x_face_69,x_face_70,x_face_71,x_face_72,x_face_73,x_face_74,x_face_75,x_face_76,x_face_77,x_face_78,x_face_79,x_face_80,x_face_81,x_face_82,x_face_83,x_face_84,x_face_85,x_face_86,x_face_87,x_face_88,x_face_89,x_face_90,x_face_91,x_face_92,x_face_93,x_face_94,x_face_95,x_face_96,x_face_97,x_face_98,x_face_99,x_face_100,x_face_101,x_face_102,x_face_103,x_face_104,x_face_105,x_face_106,x_face_107,x_face_108,x_face_109,x_face_110,x_face_111,x_face_112,x_face_113,x_face_114,x_face_115,x_face_116,x_face_117,x_face_118,x_face_119,x_face_120,x_face_121,x_face_122,x_face_123,x_face_124,x_face_125,x_face_126,x_face_127,x_face_128,x_face_129,x_face_130,x_face_131,x_face_132,x_face_133,x_face_134,x_face_135,x_face_136,x_face_137,x_face_138,x_face_139,x_face_140,x_face_141,x_face_142,x_face_143,x_face_144,x_face_145,x_face_146,x_face_147,x_face_148,x_face_149,x_face_150,x_face_151,x_face_152,x_face_153,x_face_154,x_face_155,x_face_156,x_face_157,x_face_158,x_face_159,x_face_160,x_face_161,x_face_162,x_face_163,x_face_164,x_face_165,x_face_166,x_face_167,x_face_168,x_face_169,x_face_170,x_face_171,x_face_172,x_face_173,x_face_174,x_face_175,x_face_176,x_face_177,x_face_178,x_face_179,x_face_180,x_face_181,x_face_182,x_face_183,x_face_184,x_face_185,x_face_186,x_face_187,x_face_188,x_face_189,x_face_190,x_face_191,x_face_192,x_face_193,x_face_194,x_face_195,x_face_196,x_face_197,x_face_198,x_face_199,x_face_200,x_face_201,x_face_202,x_face_203,x_face_204,x_face_205,x_face_206,x_face_207,x_face_208,x_face_209,x_face_210,x_face_211,x_face_212,x_face_213,x_face_214,x_face_215,x_face_216,x_face_217,x_face_218,x_face_219,x_face_220,x_face_221,x_face_222,x_face_223,x_face_224,x_face_225,x_face_226,x_face_227,x_face_228,x_face_229,x_face_230,x_face_231,x_face_232,x_face_233,x_face_234,x_face_235,x_face_236,x_face_237,x_face_238,x_face_239,x_face_240,x_face_241,x_face_242,x_face_243,x_face_244,x_face_245,x_face_246,x_face_247,x_face_248,...,z_face_293,z_face_294,z_face_295,z_face_296,z_face_297,z_face_298,z_face_299,z_face_300,z_face_301,z_face_302,z_face_303,z_face_304,z_face_305,z_face_306,z_face_307,z_face_308,z_face_309,z_face_310,z_face_311,z_face_312,z_face_313,z_face_314,z_face_315,z_face_316,z_face_317,z_face_318,z_face_319,z_face_320,z_face_321,z_face_322,z_face_323,z_face_324,z_face_325,z_face_326,z_face_327,z_face_328,z_face_329,z_face_330,z_face_331,z_face_332,z_face_333,z_face_334,z_face_335,z_face_336,z_face_337,z_face_338,z_face_339,z_face_340,z_face_341,z_face_342,z_face_343,z_face_344,z_face_345,z_face_346,z_face_347,z_face_348,z_face_349,z_face_350,z_face_351,z_face_352,z_face_353,z_face_354,z_face_355,z_face_356,z_face_357,z_face_358,z_face_359,z_face_360,z_face_361,z_face_362,z_face_363,z_face_364,z_face_365,z_face_366,z_face_367,z_face_368,z_face_369,z_face_370,z_face_371,z_face_372,z_face_373,z_face_374,z_face_375,z_face_376,z_face_377,z_face_378,z_face_379,z_face_380,z_face_381,z_face_382,z_face_383,z_face_384,z_face_385,z_face_386,z_face_387,z_face_388,z_face_389,z_face_390,z_face_391,z_face_392,z_face_393,z_face_394,z_face_395,z_face_396,z_face_397,z_face_398,z_face_399,z_face_400,z_face_401,z_face_402,z_face_403,z_face_404,z_face_405,z_face_406,z_face_407,z_face_408,z_face_409,z_face_410,z_face_411,z_face_412,z_face_413,z_face_414,z_face_415,z_face_

In [10]:
df = df[df.columns[~df.columns.str.contains('z_')]].copy()

In [11]:
df.index.unique()[0]

1816796431

In [12]:
df[df.index == 1816796431].shape

(123, 1087)

In [13]:
batch = []
for sequence in tqdm(df.index.unique()):
    df_sequence = df[df.index == sequence].copy()
    df_sequence = df_sequence.drop(columns ='frame')
    np_sequence = np.pad(df_sequence.T.values, ((0,0),(0,1000 - len(df_sequence))), mode='constant', constant_values=0)
    batch.append(np_sequence)

100%|█████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 405.35it/s]


In [14]:
batch[0].shape

(1086, 1000)

In [15]:
x = np.stack(batch)
x.shape

(1000, 1086, 1000)

In [16]:
import json
/home/baursafi/Documents/data/kaggle-google-sign-language

In [20]:
json_path = PATH + "character_to_prediction_index.json"
print(json_path)
with open("/home/baursafi/Documents/data/kaggle-google-sign-language/character_to_prediction_index.json", 'r') as f:
    index_map = json.load(f)

~/Documents/data/kaggle-google-sign-language/character_to_prediction_index.json


In [21]:
def string_to_array(string:str, map_index:dir):
    tokens = [map_index[letter] for letter in string]
    tokens = tokens + [0] * (40 - len(tokens))
    return tokens

In [29]:
y = df_train[df_train.file_id == 5414471].phrase.apply(lambda x: string_to_array(x, index_map))

In [30]:
y = np.stack(y)

In [40]:
import tensorflow as tf
import keras
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed
from keras import Input
from keras.models import Sequential


## Test NN

In [41]:
HEIGHT = 1086
WIDTH = 1000
print(x.shape)

(1000, 1086, 1000)


In [55]:
model = tf.keras.models.Sequential(name="GRU-model")
model.add(tf.keras.Input(shape=(x.shape[1], 
                                x.shape[2])))

#add a couple of layers
model.add(GRU(units=512, return_sequences=True, name='GRU'))
model.add(Bidirectional(GRU(units=512, 
                            activation='tanh', 
                            stateful=False), 
                        name='Hidden-GRU-Encoder-Layer')) 
model.add(RepeatVector(x.shape[1], name='repeat-vector-layer'))
model.add(Bidirectional(GRU(units=32, 
                            activation='tanh', 
                            recurrent_activation='sigmoid')))

#output layer
model.add(Dense(200, activation='softmax'))

#compilie the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#print model summary
model.summary()

2023-07-03 07:56:36.576006: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-03 07:56:36.577125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-03 07:56:36.578013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "GRU-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 GRU (GRU)                   (None, 1086, 512)         2325504   
                                                                 
 Hidden-GRU-Encoder-Layer (B  (None, 1024)             3151872   
 idirectional)                                                   
                                                                 
 repeat-vector-layer (Repeat  (None, 1086, 1024)       0         
 Vector)                                                         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               203136    
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 200)               13000     
                                                         

2023-07-03 07:56:37.402280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-03 07:56:37.403790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-03 07:56:37.405259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
history = model.fit(x, y, 
                    batch_size = 1, 
                    verbose=1, 
                    validation_split = 0.2,
                    shuffle=True)